In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# for the model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, RepeatVector, TimeDistributed, Reshape
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [114]:
# load files from pre-processing

#X = np.load('X.npy') #rainpredictions, hour of day, month of year, holiday, ones
#y = np.load('y.npy')

X_dry = np.load('X_dry.npy',allow_pickle=True)
y_dry = np.load('Y_dry.npy',allow_pickle=True)
X_rain = np.load('X_rain.npy',allow_pickle=True)
y_rain = np.load('Y_rain.npy',allow_pickle=True)
TimeHour = pd.read_csv('Timehour_wet.csv')

array([[554.1805633333325, 0.0, 0.0, ..., 0.0, 1.0, 1.0],
       [512.610779583333, 1.0, 0.0, ..., 0.0, 0.0, 1.0],
       [495.14290208333324, 0.0, 1.0, ..., 0.0, 0.0, 1.0],
       ...,
       [113.38789088512496, 0.0, 0.0, ..., 0.0, 0.0, 1.0],
       [77.43711137416668, 0.0, 0.0, ..., 0.0, 0.0, 1.0],
       [163.3501073891665, 0.0, 0.0, ..., 0.0, 0.0, 1.0]], dtype=object)

In [116]:
# split dataset in test and train

def ReshapeXData(dataframe):
    n_hours = 24
    n_features = dataframe.shape[1]
    dataframe_shaped = np.reshape(dataframe,(-1, n_hours, n_features))
    return dataframe_shaped

def ReshapeYData(dataframe):
    n_hours = 24
    n_features = 1
    dataframe_shaped = np.reshape(dataframe,(-1, n_hours, n_features))
    return dataframe_shaped

#test_size for X_rain = 0.1984 as to properly split the set into groups of 24
#test_size for X_dry = 0.2 for a proper split
X_train, X_test, y_train, y_test = train_test_split(X_rain, y_rain, test_size=0.1984, random_state=42)

X_shaped = ReshapeXData(X_rain)
X_train = ReshapeXData(X_train)
X_test = ReshapeXData(X_test)
y_train = ReshapeYData(y_train)
y_test = ReshapeYData(y_test)

#X_train_gru = X[0:10800,:]
#X_test_gru = X[10801:13609,:]
#Y_train_gru = y[0:10800]
#Y_test_gru = y[10801:13609]


#X_train_gru_shaped=ReshapeXData(X_train_gru)
#X_test_gru_shaped = ReshapeXData(X_test_gru)
#Y_train_gru_shaped = ReshapeYData(Y_train_gru)
#Y_test_gru_shaped = ReshapeYData(Y_test_gru)
#X_shaped = ReshapeXData(X[0:13608,:])
#Y_shaped = ReshapeYData(y[0:13608])

In [117]:
# basic model

from keras.layers import Dense

lr=0.0002
epochs=100
batch_size=32
validation_split=0.1

print('Build model...')

n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
#n_timesteps, n_features, n_outputs = X_train_gru.shape[0], X_train_gru.shape[1], Y_train_gru.shape[0]
# reshape output into [samples, timesteps, features]

model = Sequential()
#model.add(Reshape((1, X_train_gru.shape[1])))
model.add(GRU(200, activation='relu', input_shape=(n_timesteps, n_features)))
#model.add(GRU(200, activation='tanh', input_shape=(1,X_train_gru.shape[1])))
model.add(RepeatVector(n_outputs))
model.add(GRU(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
opt=optimizers.Adam(learning_rate=lr)
model.compile(loss='mae', optimizer=opt)

model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        validation_split=0.2,
        callbacks=[
            ReduceLROnPlateau(
                verbose=1)])
    
    
#model = Sequential()
#model.add(Dense(1, input_dim=X.shape[1]))
#model.compile(optimizer = optimizers.Adam(lr=lr), loss='mean_squared_error')
#model.fit(X_train, y_train,
#          validation_split=validation_split,
#          epochs=epochs,
#          batch_size=batch_size,
#          shuffle=False)

Build model...
Train on 84 samples, validate on 21 samples
Epoch 1/100
84/84 [==============================] - 1s 7ms/step - loss: 215.6972 - val_loss: 211.8116
Epoch 2/100
84/84 [==============================] - 0s 2ms/step - loss: 206.7284 - val_loss: 202.5882
Epoch 3/100
84/84 [==============================] - 0s 2ms/step - loss: 197.7885 - val_loss: 194.9771
Epoch 4/100
84/84 [==============================] - 0s 2ms/step - loss: 189.3910 - val_loss: 187.3199
Epoch 5/100
84/84 [==============================] - 0s 2ms/step - loss: 181.1372 - val_loss: 180.5817
Epoch 6/100
84/84 [==============================] - 0s 2ms/step - loss: 174.0649 - val_loss: 17

In [111]:
# make a prediction to test how well the model preforms
y_predicted = model.predict(X_test)

In [ ]:
### LZS: WE HAVE TO IMPLEMENT HERE SOME METRICS AND A LOT OF PLOTS SO THAT WE CAN SEE HOW WELL THE MODEL PERFORMS
### LZS: AND ALSO TO BE ABLE TO FIND NEW FEATURES TO IMPROVE THE MODEL
### LZS: AND TO EVALUATE IF THE MODEL INDEED IMPROVES WHEN ADDING MORE FEATURES/ OTHER LAYERS

In [112]:
# show the inputs, predicted outputs and real outputs
for i in range(len(X_test[0])):
        for j in range(len(X_test[1])):
            print("Predicted=%s, Original=%s" % (y_predicted[i][j], y_test[i][j]))

Predicted=[72.248436], Original=[84.4234122904167]
Predicted=[86.923], Original=[862.5090558333333]
Predicted=[88.00898], Original=[365.5664364264861]
Predicted=[88.88442], Original=[336.159522455139]
Predicted=[90.23452], Original=[146.23547872013904]
Predicted=[91.5743], Original=[63.33341904555554]
Predicted=[92.92094], Original=[498.6270929166664]
Predicted=[94.30749], Original=[514.8907244444447]
Predicted=[95.76746], Original=[513.2867190277781]
Predicted=[97.24717], Original=[43.21729329622221]
Predicted=[98.77364], Original=[35.78967036840332]
Predicted=[100.38198], Original=[120.85732251188983]
Predicted=[102.02639], Original=[35.80649369249831]
Predicted=[103.73035], Original=[10.226580554208333]
Predicted=[105.49932], Original=[91.94390274458337]
Predicted=[107.35139], Original=[83.82949924069443]
Predicted=[109.34038], Original=[104.96816865428814]
Predicted=[111.46469], Original=[496.9309556388885]
Predicted=[113.796715], Original=[151.64836002888876]
Predicted=[116.36996]

In [115]:
# Create a dataframe with a prediction for all datapoints for the csv
y_predicted_all = model.predict(X_shaped)
y_pred_all = np.reshape(y_predicted_all, (-1, 1))
print(y_pred_all.shape)
y_pred_all = pd.DataFrame(data=y_pred_all)
print(TimeHour.shape)
my_df = pd.DataFrame({'Time': TimeHour.loc[:,'TimeHour'].tolist(), 'pred_y': y_pred_all.loc[:,0].tolist()})
my_df.to_csv('WetPredictions.csv')


(3144, 1)
(3144, 2)
